## Validação do dataset

# Importando Datasets

In [8]:
import pandas as pd
import os

# --- Configuração ---
dataset_path = r'C:\Github\unicamp-tcc-engenharia-banco-de-dados\Prosposta 01 - Modelo de Machine Learning - Vendas\datasets\Retail Sales Forecasting Data'

# Função auxiliar para carregar um CSV com tratamento de erros
def load_csv_to_df(filename, base_path):
    file_path = os.path.join(base_path, filename)
    try:
        # Tenta com a codificação padrão UTF-8
        return pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip')
    except UnicodeDecodeError:
        print(f"Aviso: Usando codificação 'latin1' para o arquivo {filename}")
        # Tenta com latin1 se UTF-8 falhar
        return pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado: {file_path}")
        return None
    except Exception as e:
        print(f"ERRO ao ler {filename}: {e}")
        return None

# --- Carregamento dos Dados em variáveis separadas ---
print("Carregando cada CSV em uma variável de DataFrame dedicada...")

actual_matrix_df = load_csv_to_df('actual_matrix.csv', dataset_path)
catalog_df = load_csv_to_df('catalog.csv', dataset_path)
discounts_history_df = load_csv_to_df('discounts_history.csv', dataset_path)
markdowns_df = load_csv_to_df('markdowns.csv', dataset_path)
online_df = load_csv_to_df('online.csv', dataset_path)
price_history_df = load_csv_to_df('price_history.csv', dataset_path)
sales_df = load_csv_to_df('sales.csv', dataset_path)
stores_df = load_csv_to_df('stores.csv', dataset_path)

print("\n✅ DataFrames carregados!")

Carregando cada CSV em uma variável de DataFrame dedicada...

✅ DataFrames carregados!


### Verificando colunas

In [9]:
# --- Print do cabeçalho de todos os DataFrames carregados ---
dataframes = [
    ("actual_matrix_df", actual_matrix_df),
    ("catalog_df", catalog_df),
    ("discounts_history_df", discounts_history_df),
    ("markdowns_df", markdowns_df),
    ("online_df", online_df),
    ("price_history_df", price_history_df),
    ("sales_df", sales_df),
    ("stores_df", stores_df)
]

for name, df in dataframes:
    if df is not None:
        print(f"\n--- Cabeçalho do DataFrame '{name}' ---")
        print(df.head())
    else:
        print(f"\n--- DataFrame '{name}' não foi carregado. ---")


--- Cabeçalho do DataFrame 'actual_matrix_df' ---
   Unnamed: 0       item_id        date  store_id
0           0  c578da8e8841  2019-10-17         1
1           1  4dd2b686802c  2019-10-17         1
2           2  16ade2e3fca9  2019-10-17         1
3           3  f4eb16dab09e  2019-10-17         1
4           4  ab1384c2af6d  2020-05-31         1

--- Cabeçalho do DataFrame 'catalog_df' ---
   Unnamed: 0       item_id                 dept_name          class_name  \
0           0  da17e2d5feda  БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ  БУМАЖНАЯ ПРОДУКЦИЯ   
1           1  614de2b96018  БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ    ВАТНАЯ ПРОДУКЦИЯ   
2           2  0c1f1f3e3e11  БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ    ВАТНАЯ ПРОДУКЦИЯ   
3           3  71a7fa99f005           ТОВАРЫ ДЛЯ ДОМА              ПОСУДА   
4           4  ec1bd4d59fe9  БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ    ВАТНАЯ ПРОДУКЦИЯ   

      subclass_name item_type  weight_volume  weight_netto  fatness  
0  ВЛАЖНЫЕ САЛФЕТКИ       NaN          150.0           NaN      NaN  
1

In [10]:
display(catalog_df)

,Unnamed: 0,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
0,0,da17e2d5feda,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,БУМАЖНАЯ ПРОДУКЦИЯ,ВЛАЖНЫЕ САЛФЕТКИ,NaN,150.0,NaN,NaN
1,1,614de2b96018,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
2,2,0c1f1f3e3e11,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,NaN,NaN,NaN
3,3,71a7fa99f005,ТОВАРЫ ДЛЯ ДОМА,ПОСУДА,КУХОННАЯ ПОСУДА,NaN,NaN,NaN,NaN
4,4,ec1bd4d59fe9,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
219805,219806,a8eae2f5974a,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,ПЕЧЕНЬЕ ФАСОВАННОЕ,NaN,NaN,NaN,NaN
219806,219807,54a786e4a589,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,ПЕЧЕНЬЕ ФАСОВАННОЕ,NaN,NaN,NaN,NaN
219807,219808,a96e686cb2ff,ДЕТСКИЙ МИР,ИГРУШКИ,РАЗВИВАЮЩИЕ ИГРУШКИ,NaN,NaN,NaN,NaN
219808,219809,49cdb76c6eb7,ДЕТСКИЙ МИР,ИГРУШКИ,РАЗВИВАЮЩИЕ ИГРУШКИ,NaN,NaN,NaN,NaN


### Traduzindo datasets

Foi traduzido o dataframe catalog_df;

As colunas 'dept_name', 'class_name', 'subclass_name' estão na linguam Búlgaro.

In [11]:
from langdetect import detect
from collections import Counter

def detectar_idioma_mais_comum(coluna):
    idiomas = []
    for texto in coluna.dropna().unique():
        try:
            idiomas.append(detect(str(texto)))
        except:
            continue
    contagem = Counter(idiomas)
    idioma_mais_comum = contagem.most_common(1)[0]
    return idioma_mais_comum

# Detectar idioma das 3 colunas
for col in ['dept_name', 'class_name', 'subclass_name']:
    idioma, ocorrencias = detectar_idioma_mais_comum(catalog_df[col])
    print(f"🗂️ Coluna '{col}': idioma predominante = {idioma.upper()} ({ocorrencias} ocorrências)")


🗂️ Coluna 'dept_name': idioma predominante = BG (87 ocorrências)
🗂️ Coluna 'class_name': idioma predominante = BG (280 ocorrências)
🗂️ Coluna 'subclass_name': idioma predominante = BG (454 ocorrências)


In [12]:
display(catalog_df)

,Unnamed: 0,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
0,0,da17e2d5feda,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,БУМАЖНАЯ ПРОДУКЦИЯ,ВЛАЖНЫЕ САЛФЕТКИ,NaN,150.0,NaN,NaN
1,1,614de2b96018,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
2,2,0c1f1f3e3e11,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,NaN,NaN,NaN
3,3,71a7fa99f005,ТОВАРЫ ДЛЯ ДОМА,ПОСУДА,КУХОННАЯ ПОСУДА,NaN,NaN,NaN,NaN
4,4,ec1bd4d59fe9,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
219805,219806,a8eae2f5974a,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,ПЕЧЕНЬЕ ФАСОВАННОЕ,NaN,NaN,NaN,NaN
219806,219807,54a786e4a589,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,ПЕЧЕНЬЕ ФАСОВАННОЕ,NaN,NaN,NaN,NaN
219807,219808,a96e686cb2ff,ДЕТСКИЙ МИР,ИГРУШКИ,РАЗВИВАЮЩИЕ ИГРУШКИ,NaN,NaN,NaN,NaN
219808,219809,49cdb76c6eb7,ДЕТСКИЙ МИР,ИГРУШКИ,РАЗВИВАЮЩИЕ ИГРУШКИ,NaN,NaN,NaN,NaN


### Realizando tradução

In [13]:
import pandas as pd
from deep_translator import GoogleTranslator, MyMemoryTranslator
from langdetect import detect
import time
import requests
import warnings

# Ignora avisos de SSL temporariamente (use com cuidado)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Função para detectar idioma
def detect_language(text):
    try:
        if isinstance(text, str) and text.strip():
            return detect(text)
        return None
    except:
        return None

# Função para traduzir texto com fallback
def translate_text(text, source='auto', target='pt'):
    if not isinstance(text, str) or not text.strip():
        return text  # Retorna texto original se não for string válida
    
    # Detecta idioma se source='auto'
    if source == 'auto':
        source = detect_language(text) or 'ru'  # Assume russo como padrão se detecção falhar
    
    try:
        # Tenta Google Translate
        translated = GoogleTranslator(source=source, target=target).translate(text)
        return translated
    except Exception as e:
        print(f"Erro no Google Translate para '{text}': {e}")
        try:
            # Fallback para MyMemoryTranslator
            print(f"Tentando MyMemoryTranslator para '{text}'...")
            translated = MyMemoryTranslator(source=source, target=target).translate(text)
            return translated
        except Exception as e2:
            print(f"Erro no MyMemoryTranslator para '{text}': {e2}")
            return text  # Retorna texto original em caso de erro

# Função para traduzir valores únicos de uma coluna
def translate_column(df, column_name):
    print(f"Traduzindo coluna '{column_name}'...")
    # Obtém valores únicos da coluna
    unique_values = df[column_name].dropna().unique()
    # Cria um dicionário de tradução para valores únicos
    translation_map = {val: translate_text(val, source='auto') for val in unique_values}
    
    # Aplica o mapeamento à coluna
    df[column_name] = df[column_name].map(translation_map).fillna(df[column_name])
    return df

# Verifica se catalog_df foi carregado
if catalog_df is not None:
    print("Iniciando tradução das colunas 'dept_name', 'class_name' e 'subclass_name'...")
    
    # Lista de colunas a traduzir
    columns_to_translate = ['dept_name', 'class_name', 'subclass_name']
    
    # Verifica se as colunas existem no DataFrame
    missing_columns = [col for col in columns_to_translate if col not in catalog_df.columns]
    if missing_columns:
        print(f"ERRO: As seguintes colunas não foram encontradas em catalog_df: {missing_columns}")
    else:
        # Traduz cada coluna
        for column in columns_to_translate:
            catalog_df = translate_column(catalog_df, column)
            # Pausa para evitar sobrecarga na API
            time.sleep(2)
        
        print("\n✅ Tradução concluída!")
        
        # Exibe o cabeçalho do catalog_df após tradução
        print("\n--- Cabeçalho do DataFrame 'catalog_df' após tradução ---")
        print(catalog_df.head())
        
        # Opcional: Salva o DataFrame traduzido
        catalog_df.to_csv(os.path.join(dataset_path, 'catalog_translated.csv'), index=False)
else:
    print("ERRO: catalog_df não foi carregado. Verifique o carregamento do arquivo 'catalog.csv'.")

Iniciando tradução das colunas 'dept_name', 'class_name' e 'subclass_name'...
Traduzindo coluna 'dept_name'...
Traduzindo coluna 'class_name'...
Traduzindo coluna 'subclass_name'...

✅ Tradução concluída!

--- Cabeçalho do DataFrame 'catalog_df' após tradução ---
   Unnamed: 0       item_id          dept_name           class_name  \
0           0  da17e2d5feda  Produtos de papel              Teimoso   
1           1  614de2b96018  Produtos de papel  Produtos de algodão   
2           2  0c1f1f3e3e11  Produtos de papel  Produtos de algodão   
3           3  71a7fa99f005     Carrega a casa                Prato   
4           4  ec1bd4d59fe9  Produtos de papel  Produtos de algodão   

           subclass_name item_type  weight_volume  weight_netto  fatness  
0   Guardanapos molhados       NaN          150.0           NaN      NaN  
1      Discos de algodão       NaN           30.0           NaN      NaN  
2      Discos de algodão       NaN            NaN           NaN      NaN  
3  Utensí

In [20]:
display(catalog_df)

,Unnamed: 0,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
0,0,da17e2d5feda,Produtos de papel,Teimoso,Guardanapos molhados,NaN,150.0,NaN,NaN
1,1,614de2b96018,Produtos de papel,Produtos de algodão,Discos de algodão,NaN,30.0,NaN,NaN
2,2,0c1f1f3e3e11,Produtos de papel,Produtos de algodão,Discos de algodão,NaN,NaN,NaN,NaN
3,3,71a7fa99f005,Carrega a casa,Prato,Utensílios de cozinha,NaN,NaN,NaN,NaN
4,4,ec1bd4d59fe9,Produtos de papel,Produtos de algodão,Discos de algodão,NaN,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
219805,219806,a8eae2f5974a,Doces,Barriga,Baste jejum,NaN,NaN,NaN,NaN
219806,219807,54a786e4a589,Doces,Barriga,Baste jejum,NaN,NaN,NaN,NaN
219807,219808,a96e686cb2ff,A PAZ DE DECIDI,Jogador,Jogador,NaN,NaN,NaN,NaN
219808,219809,49cdb76c6eb7,A PAZ DE DECIDI,Jogador,Jogador,NaN,NaN,NaN,NaN


## Download do dataset traduzido para portugues

In [15]:
import os

# Define o caminho para salvar o arquivo CSV
output_path = os.path.join(dataset_path, 'catalog_translated_pt_br.csv')

# Salva o DataFrame como CSV
catalog_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Arquivo salvo com sucesso em: {output_path}")

✅ Arquivo salvo com sucesso em: C:\Github\unicamp-tcc-engenharia-banco-de-dados\Prosposta 01 - Modelo de Machine Learning - Vendas\datasets\Retail Sales Forecasting Data\catalog_translated_pt_br.csv


In [16]:
# Função para verificar traduções
def check_translations(df, columns):
    print("\n--- Verificação de Traduções ---")
    for column in columns:
        print(f"\nColuna: {column}")
        # Obtém valores únicos
        unique_values = df[column].dropna().unique()
        # Identifica valores que parecem não estar traduzidos (contêm cirílico)
        cyrillic_values = [val for val in unique_values if isinstance(val, str) and any(ord(c) > 127 for c in val)]
        print(f"Total de valores únicos: {len(unique_values)}")
        print(f"Valores únicos: {unique_values[:10]}")  # Mostra até 10 valores para brevidade
        if cyrillic_values:
            print(f"⚠️ Valores possivelmente não traduzidos (contêm cirílico): {cyrillic_values}")
        else:
            print("✅ Nenhum valor em cirílico detectado (parece tudo traduzido).")

# Colunas a verificar
columns_to_check = ['dept_name', 'class_name', 'subclass_name']

# Executa a verificação
if catalog_df is not None:
    check_translations(catalog_df, columns_to_check)
else:
    print("ERRO: catalog_df não está carregado.")


--- Verificação de Traduções ---

Coluna: dept_name
Total de valores únicos: 193
Valores únicos: ['Produtos de papel' 'Carrega a casa' 'Produtos para o harém'
 'Refrescadores, inseticidas' 'Tigres do Confet'
 'Chocolad, pasta de chocolate' 'Pão' 'Caviar de peixe rio e mar'
 'Peixe para cerveja' 'Rybki k pivo (não morda)']
⚠️ Valores possivelmente não traduzidos (contêm cirílico): ['Produtos para o harém', 'Pão', 'Rybki k pivo (não morda)', 'Carne, pássaro congelado', 'Bebidas com baixo teor de álcool', 'Crepkie bebidas alcoólicas', 'Carrega ataca pessoalmente o útil', 'Amanhã', 'Dano a produtos lácteos', 'Fala, referência', 'Preservação', 'Café', 'Produtos dabéticos', 'Questão dietética', 'Doçura', 'Ingredientes são padaria', 'Caviar de salmão', 'Pássaros', 'CHÁ', 'Petróleo e margarina', 'Acessórios para fumar', 'Automático', 'Produtos rápidos -feitos', 'ÁGUA', 'Cargas e escritório', 'Catálogo Global', 'Interlocação', 'Lealdade de ação (não extermine)', 'As bagas estão congeladas', 'C

## Carregando dados traduzidos

In [18]:
import os

output_path = os.path.join(dataset_path, 'catalog_translated.csv')
if os.path.exists(output_path):
    print(f"✅ Arquivo 'catalog_translated.csv' encontrado em: {output_path}")
    # Opcional: Exibe as primeiras linhas do arquivo salvo
    saved_df = pd.read_csv(output_path, encoding='utf-8')
    print("\n--- Primeiras linhas do arquivo salvo ---")
    print(saved_df.head())
else:
    print(f"⚠️ Arquivo 'catalog_translated.csv' não encontrado. Tentando salvar novamente...")
    catalog_df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ Arquivo salvo novamente em: {output_path}")

✅ Arquivo 'catalog_translated.csv' encontrado em: C:\Github\unicamp-tcc-engenharia-banco-de-dados\Prosposta 01 - Modelo de Machine Learning - Vendas\datasets\Retail Sales Forecasting Data\catalog_translated.csv

--- Primeiras linhas do arquivo salvo ---
   Unnamed: 0       item_id          dept_name           class_name  \
0           0  da17e2d5feda  Produtos de papel              Teimoso   
1           1  614de2b96018  Produtos de papel  Produtos de algodão   
2           2  0c1f1f3e3e11  Produtos de papel  Produtos de algodão   
3           3  71a7fa99f005     Carrega a casa                Prato   
4           4  ec1bd4d59fe9  Produtos de papel  Produtos de algodão   

           subclass_name item_type  weight_volume  weight_netto  fatness  
0   Guardanapos molhados       NaN          150.0           NaN      NaN  
1      Discos de algodão       NaN           30.0           NaN      NaN  
2      Discos de algodão       NaN            NaN           NaN      NaN  
3  Utensílios de co

## Verificando e Corrigindo as Traduções

In [29]:
import pandas as pd
import re
import os

# Função para verificar se o texto contém caracteres cirílicos
def has_cyrillic(text):
    if not isinstance(text, str):
        return False
    return bool(re.search(r'[\u0400-\u04FF]', text))

# Mapeamento manual para termos problemáticos e suspeitos
manual_translations = {
    'Produtos para o harém': 'Produtos para casa',
    'Rybki k pivo (não morda)': 'Petiscos de peixe para cerveja',
    'Crepkie bebidas alcoólicas': 'Bebidas alcoólicas fortes',
    'CHÁ': 'Chá',
    'ÁGUA': 'Água',
    'Confetы, chocolad, doçura': 'Doces, chocolate, confeitos',
    'Macaronы': 'Macarrão',
    'Produtos para гарема': 'Produtos para casa',
    'Produto marinhoы': 'Produtos marinhos',
    'Mannы': 'Sêmola',
    'Marinadы': 'Marinadas',
    'Mыlo': 'Sabonete',
    'Maasnы': 'Óleos',
    'Maaso productsы': 'Produtos de óleo',
    'Carrega a casa': 'Produtos para casa',
    'Tigres do Confet': 'Doces de confeito',
    'Chocolad, pasta de chocolate': 'Chocolate, pasta de chocolate',
    'Porcos caviar': 'Caviar de peixe',
    'Refrescadores, inseticidas': 'Repelentes e inseticidas',
    'Ladrilhos de chocolate': 'Tabletes de chocolate',
    'Patnant': 'Produtos especiais',
    'Chocolad 80-100 grama': 'Chocolate 80-100 gramas',
    'Refrescador': 'Repelente',
    'Teimoso': 'Papel higiênico',
    'Necessário': 'Produtos essenciais',
    'Pelo fabricante': 'Por fabricante',
    'G/b': 'Gramas por unidade',
}

# Função para retraduzir valores com cirílico ou incorretos
def retranslate_column(df, column_name):
    print(f"Retraduzindo coluna '{column_name}'...")
    # Identifica valores com cirílico ou no mapeamento manual
    cyrillic_values = df[column_name][df[column_name].apply(lambda x: isinstance(x, str) and (has_cyrillic(x) or x in manual_translations))].unique()
    if len(cyrillic_values) == 0:
        print(f"✅ Nenhum valor com cirílico ou problemático encontrado em '{column_name}'.")
        return df
    
    # Cria mapeamento de retradução
    translation_map = {}
    untranslated_values = []
    suspicious_values = []
    for val in cyrillic_values:
        if val in manual_translations:
            translation_map[val] = manual_translations[val]
        else:
            translation_map[val] = val  # Mantém o valor se não estiver no mapeamento
            if has_cyrillic(val):
                untranslated_values.append(val)
            # Flag valores suspeitos
            if any(keyword in val.lower() for keyword in ['carrega', 'tigres', 'chocolad', 'porcos', 'refrescadores', 'ladrilhos', 'patnant', 'teimoso', 'necessário', 'fabricante', 'g/b']):
                suspicious_values.append(val)
    
    # Aplica o mapeamento
    df[column_name] = df[column_name].map(translation_map).fillna(df[column_name])
    print(f"Valores retraduzidos em '{column_name}': {translation_map}")
    if untranslated_values:
        print(f"⚠️ Valores não traduzidos (ainda com cirílico): {untranslated_values}")
    if suspicious_values:
        print(f"⚠️ Valores potencialmente mal traduzidos (revisar): {suspicious_values}")
    return df

# Colunas a retraduzir
columns_to_retranslate = ['dept_name', 'class_name', 'subclass_name']

# Caminho do dataset
dataset_path = r'C:\Github\unicamp-tcc-engenharia-banco-de-dados\Prosposta 01 - Modelo de Machine Learning - Vendas\datasets\Retail Sales Forecasting Data'

# Verifica se catalog_df está carregado; se não, carrega do CSV
if 'catalog_df' not in globals():
    input_path = os.path.join(dataset_path, 'catalog_translated.csv')
    if os.path.exists(input_path):
        print(f"Carregando catalog_df de: {input_path}")
        catalog_df = pd.read_csv(input_path, encoding='utf-8')
    else:
        print(f"ERRO: Arquivo {input_path} não encontrado.")
        exit()

# Inicia a retradução
print("Iniciando retradução das colunas com valores problemáticos...")

# Retraduz cada coluna
for column in columns_to_retranslate:
    catalog_df = retranslate_column(catalog_df, column)

print("\n✅ Retradução concluída!")

# Exibe o cabeçalho do catalog_df após retradução
print("\n--- Cabeçalho do DataFrame 'catalog_df' após retradução ---")
print(catalog_df.head())

# Verifica novamente valores com cirílico
print("\n--- Verificação Final de Traduções ---")
for column in columns_to_retranslate:
    cyrillic_values = catalog_df[column][catalog_df[column].apply(lambda x: isinstance(x, str) and has_cyrillic(x))].unique()
    print(f"\nColuna: {column}")
    print(f"Valores únicos (amostra): {catalog_df[column].dropna().unique()[:10]}")
    if cyrillic_values.size > 0:
        print(f"⚠️ Valores ainda com cirílico: {cyrillic_values}")
    else:
        print("✅ Nenhum valor com cirílico detectado.")

# Salva o DataFrame retraduzido
output_path = os.path.join(dataset_path, 'catalog_translated_final.csv')
catalog_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Arquivo retraduzido salvo em: {output_path}")

Iniciando retradução das colunas com valores problemáticos...
Retraduzindo coluna 'dept_name'...
✅ Nenhum valor com cirílico ou problemático encontrado em 'dept_name'.
Retraduzindo coluna 'class_name'...
✅ Nenhum valor com cirílico ou problemático encontrado em 'class_name'.
Retraduzindo coluna 'subclass_name'...
✅ Nenhum valor com cirílico ou problemático encontrado em 'subclass_name'.

✅ Retradução concluída!

--- Cabeçalho do DataFrame 'catalog_df' após retradução ---
   Unnamed: 0       item_id           dept_name           class_name  \
0           0  da17e2d5feda   Produtos de papel      Papel higiênico   
1           1  614de2b96018   Produtos de papel  Produtos de algodão   
2           2  0c1f1f3e3e11   Produtos de papel  Produtos de algodão   
3           3  71a7fa99f005  Produtos para casa                Prato   
4           4  ec1bd4d59fe9   Produtos de papel  Produtos de algodão   

           subclass_name item_type  weight_volume  weight_netto  fatness  
0   Guardanapos 